# 지점별로 선형모델 생성하기

In [90]:
import numpy as np
from sklearn.model_selection import KFold
import tensorflow as tf
from sklearn.metrics import mean_squared_error
import os
import pandas as pd

In [ ]:
tf.__version__

## 1-1 입출력 데이터셋 준비

In [ ]:
pwd

### 예측 데이터 concat, label 설정

In [93]:
def concat_by_month(directory_path):
    """
    CSV 파일을 읽어와 월별로 데이터를 분류하고 병합
    모든 월의 데이터를 하나의 데이터프레임으로 return, 각 행은 월 정보를 포함

    :param directory_path: CSV 파일이 저장된 디렉토리 경로
    :return: 하나의 데이터프레임으로 병합된 월별 데이터
    """
    # 파일 목록을 읽어온다
    file_list = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

    # 월별 데이터를 저장할 딕셔너리 초기화
    monthly_data = {str(i): [] for i in range(1, 13)}

    # 파일명에서 날짜를 파싱하여 해당 월에 맞는 데이터 프레임 리스트에 추가
    for file_name in file_list:
        date_part = file_name.split('_')[4]  # '20220614' 형식의 문자열 추출
        month = date_part[4:6]  # 월 정보 추출
        
        if month[0] == '0':
            month = month[1]  # '01'과 같은 경우 '1'로 변환
        
        # 파일 읽기
        file_path = os.path.join(directory_path, file_name)
        df = pd.read_csv(file_path)
        df = df[:-1]
        df['Month'] = month  # 월 정보를 데이터프레임에 추가

        # 해당 월에 데이터프레임 추가
        monthly_data[month].append(df)
        
    # 모든 월별 데이터를 하나의 데이터프레임으로 병합
    month_order = ['6', '7', '8', '9', '10', '11', '12', '1', '2', '3', '4', '5']
    all_months_df = pd.concat([pd.concat(monthly_data[month], ignore_index=True) for month in month_order if monthly_data[month]], ignore_index=True)
    
    return all_months_df

In [94]:
month_df = concat_by_month('../south3')

In [ ]:
month_df

### 관측데이터 label 설정

In [96]:
def create_month_column(filename):
    """
    현재 디렉토리에서 지정된 파일을 읽어 'obs_time' 열을 파싱하여 월 정보를 추출하고
    새로운 'Month' 열을 데이터프레임에 추가한 후 반환합니다.

    :param filename: 읽을 파일의 이름
    :return: 'Month' 열이 추가된 데이터프레임
    """
    # 현재 디렉토리 경로 설정
    current_directory = os.getcwd()
    # 파일 경로 완성
    file_path = os.path.join(current_directory, filename)
    
    # 파일이 존재하는지 확인
    if os.path.exists(file_path):
        # 파일 읽기
        df = pd.read_csv(file_path)
        # 'obs_time' 열이 있는지 확인 후 처리
        if 'obs_time' in df.columns:
            # 'obs_time' 열에서 날짜를 파싱하여 월 정보 추출
            df['Month'] = pd.to_datetime(df['obs_time']).dt.month
            return df
        else:
            raise ValueError("The file does not contain an 'obs_time' column.")
    else:
        raise FileNotFoundError("The specified file does not exist in the current directory.")

# 사용 예:
# filename = 'your_data.csv'
# updated_data = create_month_column_from_obs_time(filename)
# print(updated_data.head())  # 'obs_time'과 'Month' 열을 포함하는 데이터프레임 출력


In [97]:
obs_df = create_month_column('../../RnD_Ensemble/south_obs_KG_0025.csv')

In [ ]:
obs_df

### 입출력 데이터 프레임 생성

In [ ]:
obs_df.columns

In [ ]:
month_df.columns

In [101]:
def create_input_output_data(df_obs, df_pred, start_date, end_date):
    """
    파라미터:
    - df_obs (pd.DataFrame): 관측 데이터를 포함하는 데이터프레임
    - df_pred (pd.DataFrame): 예측 데이터를 포함하는 데이터프레임
    - start_date (str 또는 pd.Timestamp): 데이터 선택을 시작할 날짜 ('YYYY-MM-DD' 형식).
    - end_date (str 또는 pd.Timestamp): 데이터 선택을 종료할 날짜 ('YYYY-MM-DD' 형식).

    반환 값:
    - input_df (pd.DataFrame): 관측값과 예측값이 결합된 (96시간씩) 데이터 프레임
    - output_df (pd.DataFrame): 관측 데이터 (48시간씩) 출력 데이터 프레임
    """
    
    # 시간 열을 datetime 형식으로 변환
    df_obs['obs_time'] = pd.to_datetime(df_obs['obs_time'])
    df_pred['obs_time'] = pd.to_datetime(df_pred['model_fct']) # model_fct를 통일성을 위해 obs_time 이름으로 사용
    
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # input, ouput dataframes
    input_frames = []
    output_frames = []
    
    start_index = 0
    start_index2 = 0
    # 날짜별로 데이터 프레임 구성
    for single_date in pd.date_range(start_date, end_date):
        
        # 관측값은 전일 0h-23h만 선택

        # 시작 인덱스로부터 24개의 행을 슬라이싱
        obs_data_yesterday = df_obs.loc[start_index:start_index + 23, ['obs_time', 'obs_u', 'obs_v']].copy()
        obs_data_yesterday.rename(columns={'obs_u': 'u', 'obs_v': 'v'}, inplace=True)
        
        start_index += 24
        # 'type' 컬럼을 추가하고 'obs'로 설정 (데이터 확인용)
        obs_data_yesterday['type'] = 'obs'
        
        # 예측값은 당일에서 24~71h 후
        # single_date에 해당하는 인덱스를 찾기

        # 시작 인덱스로부터 72개의 행을 슬라이싱
        pred_data_range = df_pred.loc[start_index2:start_index2 + 71, ['obs_time', 'model_u', 'model_v']].copy()
        start_index2 += 72
        
        # 열 이름 변경 및 'type' 컬럼 추가 (데이터 확인용)
        pred_data_range.rename(columns={'model_u': 'u', 'model_v': 'v'}, inplace=True)
        pred_data_range['type'] = 'model'
        if not pred_data_range.empty and len(pred_data_range) >= 25:
            month_value = pred_data_range['obs_time'].iloc[24].month
            pred_data_range['month'] = int(month_value)
            obs_data_yesterday['month'] = int(month_value)
        elif not pred_data_range.empty:
            pred_data_range['month'] = pred_data_range['obs_time'].dt.month.iloc[0]
        else:
            continue  # 데이터가 없으면 이번 루프는 건너뛰기
        
        # input 데이터는 관측값과 예측값을 병합 (관측치 24시간 + 예측치 72시간)
        combined_input = pd.concat([obs_data_yesterday, pred_data_range])

        
        # output 데이터는 다음날 24h부터 내일 모레까지 71h의 관측값
        output_data = df_obs[
            (df_obs['obs_time'] >= single_date + pd.DateOffset(days=1)) & 
            (df_obs['obs_time'] < single_date + pd.DateOffset(days=3))
        ][['obs_time', 'obs_u', 'obs_v']].copy()
        output_data.rename(columns={'obs_u': 'u', 'obs_v': 'v'}, inplace=True)
        output_data['month'] = output_data['obs_time'].iloc[0].month

        
        input_frames.append(combined_input)
        output_frames.append(output_data)

    
    input_df = pd.concat(input_frames, ignore_index=True)
    output_df = pd.concat(output_frames, ignore_index=True)
    
    # 모든 데이터 프레임에서 'obs_time' 열 이름을 'time'으로 변경
    input_df.rename(columns={'obs_time': 'time'}, inplace=True)
    output_df.rename(columns={'obs_time': 'time'}, inplace=True)

    return input_df, output_df


In [102]:
# 입력 및 출력 데이터 생성
input_df, output_df = create_input_output_data(obs_df, month_df, '2022-06-01', '2023-05-31')

In [ ]:
input_df[:96]

In [ ]:
# 출력 데이터는 관측치 48개씩
output_df[:48]

### 96(192), 48(96)개씩 데이터 잘라서 입출력 생성

In [105]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [106]:
# 시간, u,v, month 열만 사용
input = input_df[['time', 'u', 'v', 'month']]
output = output_df[['time', 'u', 'v', 'month']]

In [ ]:
input[:26]

In [108]:
input.to_csv('input.csv')

In [ ]:
output

In [110]:
# time열을 input, ouput 데이터프레임의 인덱스로 지정
columns = ['u', 'v', 'month']

input_feat = input_df[columns]
input_feat.index = input_df['time']

output_feat = output_df[columns]
output_feat.index = output_df['time']

In [ ]:
input_feat[96*3+23:96*3+50]

In [ ]:
output_feat

In [113]:
# DataFrame을 numpy 배열로 변환
input_array = input_feat.values
output_array = output_feat.values

In [114]:
# 배열별로 결측치 세기
nan_count_in_input = np.sum(np.isnan(input_array))
nan_count_in_output = np.sum(np.isnan(output_array))

In [ ]:
nan_count_in_input

In [ ]:
nan_count_in_output

In [ ]:
input_array.shape

In [ ]:
output_array.shape

In [ ]:
len(input_array)

In [ ]:
len(output_array)

In [ ]:
input_array[94*3:94*4]

In [ ]:
output_array[:2]

### 샘플 생성 및 nan 제거

In [123]:
def simple_slice_data(input_data, output_data, input_window, output_window):
    """
    파라미터:
    - input_data (np.array): 모델 입력을 위한 원본 데이터 배열
    - output_data (np.array): 모델 출력(예측 대상)을 위한 원본 데이터 배열
    - input_window (int): 한 번에 입력으로 사용할 데이터 포인트의 수
    - output_window (int): 한 번에 출력으로 사용할 데이터 포인트의 수

    반환 값:
    - X (np.array): 입력 데이터 슬라이스의 배열.
    - y (np.array): 출력 데이터 슬라이스의 배열.
    """
    
    X = []
    y = []

    # 데이터셋의 최대 인덱스를 계산
    max_index = len(output_data) // 48
    print("max_index: {}".format(max_index))
    
    for i in range(max_index):
        
        # 입출력 데이터 슬라이스
        input_slice = input_data[i * input_window: (i + 1) * input_window]
        output_slice = output_data[i * output_window: (i + 1) * output_window]

        X.append(input_slice)
        y.append(output_slice)

    # 배열로 변환
    X = np.array(X)
    y = np.array(y)

    return X, y


In [ ]:
# 입력은 96개씩 행이 한 샘플, 출력은 48개씩 행이 한 샘플 (하루치)
input_window = 96
output_window = 48
X, y = simple_slice_data(input_array, output_array, input_window, output_window)

print("X shape:", X.shape)  # (N, 96, 3)
print("y shape:", y.shape)  # (N, 48, 3)

In [125]:
# 입출력에 있는 결측치 세기
X_nan = np.sum(np.isnan(X))
y_nan = np.sum(np.isnan(y))

In [ ]:
X_nan

In [ ]:
y_nan

In [ ]:
len(X)

In [ ]:
len(y)

In [ ]:
# 입력 데이터에서 결측치 위치 찾기
nan_indices_input = np.unique(np.where(np.isnan(X))[0])
nan_indices_input

In [ ]:
# 출력 데이터에서 결측치 위치 찾기
nan_indices_output = np.unique(np.where(np.isnan(y))[0])
nan_indices_output

In [132]:
# 두 배열을 합쳐서 중복된 값 제거
common_nan_indices = np.unique(np.concatenate([nan_indices_input, nan_indices_output]))

In [ ]:
nan_indices_input

In [ ]:
nan_indices_output

In [135]:
# 두 배열을 합쳐서 중복된 값 제거
common_nan_indices = np.unique(np.concatenate([nan_indices_input, nan_indices_output]))

In [ ]:
# input, output 샘플에서 통째로 날려야하는 n번째 샘플들
len(common_nan_indices)

In [ ]:
common_nan_indices

In [138]:
# input보다 큰 샘플 개수 삭제
valid_indices = common_nan_indices[common_nan_indices < len(X)]

In [ ]:
len(valid_indices)

In [ ]:
# input, ouput에서 common_nan_indices에 나열된 인덱스의 샘플들을 삭제
cleaned_X = np.delete(X, valid_indices, axis=0)
cleaned_y = np.delete(y, valid_indices, axis=0)

# 제거되고 남은 샘플 개수 확인
print(cleaned_X.shape)
print(cleaned_y.shape)

In [141]:
# 배열 전체에서 NaN 값 확인
nan_exists = np.isnan(cleaned_y).any()

In [ ]:
nan_exists

In [ ]:
print(cleaned_X.shape)

In [ ]:
cleaned_X[0]

In [ ]:
cleaned_y.shape

In [ ]:
import numpy as np
import pandas as pd

# cleaned_X_reshaped와 cleaned_y_reshaped 배열이 주어졌다고 가정
# 예시 데이터 생성을 위한 코드는 생략합니다. 실제 데이터 사용을 가정

# 각 시간 단계를 별도의 행으로 변환하고, 열은 u, v, month로 설정
# cleaned_X_reshaped.reshape(-1, 3)는 모든 시간 단계를 행으로, 특성을 열로 펼칩니다.
df_X = pd.DataFrame(cleaned_X.reshape(-1, 3), columns=['u', 'v', 'month'])
df_y = pd.DataFrame(cleaned_y.reshape(-1, 3), columns=['u', 'v', 'month'])

# 'month' 열을 정수형으로 변환
df_X['month'] = df_X['month'].astype(int)
df_y['month'] = df_y['month'].astype(int)

# DataFrame을 CSV 파일로 저장
df_X.to_csv("X.csv", index=False)
df_y.to_csv("y.csv", index=False)

# 저장된 파일 확인 메시지
print("Data has been saved to cleaned_X_reshaped.csv and cleaned_y_reshaped.csv.")


## 월별인덱스 확인 및 정규화

In [147]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, Reshape

In [ ]:
# 월별로 인덱스를 분할할 리스트 초기화
monthly_indices = [[] for _ in range(12)]

total = 0
# 월별로 인덱스를 분할
for month in range(1, 13):
    indices = [i for i in range(cleaned_y.shape[0]) if int(cleaned_y[i, 0, 2]) == month]
    
    # 중복 제거 및 리스트로 변환
    total += len(indices)
    print("{0}월의 인덱스: {1}".format(month, indices))
    print("{0}월의 인덱스 개수: {1}".format(month,len(indices)))
    print('================================')
    monthly_indices[month-1].append(indices)

print("전체 index 개수: {}개".format(total))

# K-폴드 교차 검증 설정
kf = KFold(n_splits=12)

In [ ]:
cleaned_X.shape

In [ ]:
cleaned_y.shape

## feature 4로 X 분리

In [ ]:
# 각 24시간 구간을 4개의 feature로 나누기 위해 reshape
# (batch_size, total_time_steps, features) -> (batch_size, 4, time_steps_per_feature, original_features)
reshaped_X = cleaned_X.reshape((cleaned_X.shape[0], 4, 24, cleaned_X.shape[2]))

# 각 시간 구간별로 feature명을 지정하고 데이터프레임으로 변환 후 numpy 배열로 변환
numpy_arrays = []
for i in range(reshaped_X.shape[1]):  # 4개의 24시간 구간
    data = reshaped_X[:, i, :, :].reshape(-1, reshaped_X.shape[3])
    df = pd.DataFrame(data, columns=[f"feature_{j+1}" for j in range(reshaped_X.shape[3])])
    numpy_array = df.to_numpy()  # 데이터프레임을 numpy 배열로 변환
    numpy_arrays.append(numpy_array)
    print(f"Numpy array for time segment {i+1}:")
    print(numpy_array)
    print("\n")

# 각 시간 구간에 대한 numpy 배열
feature1, feature2, feature3, feature4 = numpy_arrays

# feature들을 24시간씩 잘라서 (sample_size, 24, 3) 형태로 변환
def reshape_features(feature):
    num_samples = feature.shape[0] // 24
    reshaped_feature = feature[:num_samples * 24].reshape(num_samples, 24, feature.shape[1])
    return reshaped_feature

feature1 = reshape_features(feature1)
feature2 = reshape_features(feature2)
feature3 = reshape_features(feature3)
feature4 = reshape_features(feature4)

In [ ]:
# 마지막 차원의 크기를 2로 변경 (예시로 첫 번째와 두 번째 feature만 사용)
reshaped_X = reshaped_X[:, :, :, :2]
reshaped_X.shape

In [ ]:
reshaped_X

In [154]:
X = np.concatenate((feature1, feature2, feature3, feature4), axis=2)

In [ ]:
X.shape

In [156]:
X = np.delete(X, [2,5,8,11], axis=2)

In [157]:
import pandas as pd

# 샘플 하나 선택
sample = X[0]  # 첫 번째 샘플 선택, shape=(24, 8)

# DataFrame으로 변환
df = pd.DataFrame(sample)


In [ ]:
df

In [ ]:
X

In [ ]:
cleaned_y.shape

In [161]:
# 3번째 열을 제거
y_processed = cleaned_y[:, :, :2]

In [ ]:
y_processed.shape

In [ ]:
X.shape

# 2. 모델 성능 측정

## 시간대별 (24~71h) 데이터 정리 및 저장

In [164]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, PReLU, Dense, AveragePooling1D, MaxPooling1D, Dropout, BatchNormalization, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Dense, Reshape
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
import itertools
from tensorflow.keras.initializers import glorot_normal
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import LeakyReLU ,Lambda, Activation, Bidirectional,GlobalMaxPooling1D
from keras.utils import plot_model
from tensorflow.keras.models import Model


In [165]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import font_manager, rc

plt.rc('font', family='NanumGothic')

def plot_loss_and_rmse_graphs(history, test_loss, month):
    """
    주어진 history 객체를 사용하여 훈련 및 검증 손실(MSE) 그래프와 RMSE 그래프를 로그 스케일로 그립니다.
    """

    plt.figure(figsize=(12, 6))

    # 훈련 및 검증 손실 추이 그래프 (MSE)
    plt.subplot(121)
    plt.plot(history.history['loss'], label='Training Loss (MSE)')
    plt.plot(history.history['val_loss'], label='Validation Loss (MSE)')
    plt.axhline(y=test_loss, color='r', linestyle='-', label='Test Loss (MSE)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE)')
    plt.legend(loc='upper right')
    plt.title(f'Model Loss (MSE) - Month {month+1}')
    plt.grid(True)
    plt.tight_layout()
    
    # RMSE 값 계산
    train_rmse = [round(np.sqrt(x), 4) for x in history.history['loss']]
    val_rmse = [round(np.sqrt(x), 4) for x in history.history['val_loss']]
    test_rmse = round(np.sqrt(test_loss), 4)
    
    # 훈련 및 검증 RMSE 그래프
    plt.subplot(122)
    plt.plot(train_rmse, label='Training RMSE')
    plt.plot(val_rmse, label='Validation RMSE')
    plt.axhline(y=test_rmse, color='r', linestyle='-', label='Test RMSE')
    plt.xlabel('Epoch')
    plt.ylabel('RMSE')
    plt.legend(loc='upper right')
    plt.title(f'Model RMSE - Month {month+1}')
    plt.grid(True)
    plt.tight_layout()
    
    plt.show()


In [187]:
def build_regression_model(input_steps, features, output_steps, output_features, CONV_WIDTH, STRIDE):
    inputs = Input(shape=(input_steps, features))
    x = Conv1D(filters=32, kernel_size=CONV_WIDTH, strides=STRIDE, activation='elu')(inputs)
    x = Dropout(0.4)(x)
    
    residual = Flatten()(x)
    dense_out = Dense(units=output_steps * output_features, activation='linear')(residual)
    outputs = Reshape((output_steps, output_features))(dense_out)
    
    # Functional API로 모델 생성
    model = Model(inputs=inputs, outputs=outputs)
    
    optimizer = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

    return model

In [188]:
from IPython.display import Image
from sklearn.preprocessing import RobustScaler

def calculate_and_store_results(cleaned_X, cleaned_y, unique_monthly_indices):
    # 결과를 저장할 딕셔너리 초기화
    results_dict = {i: {'predictions': [], 'true_values': []} for i in range(48)}
    all_results = []  # 모든 결과를 담기 위한 리스트
    all_loss = []
    all_val_loss = []
    
    # 모든 폴드에 대한 손실 값을 저장할 리스트
    fold_train_losses = []
    fold_val_losses = []
    fold_test_losses = []
    fold_rmse_values = []
    
    # 12-fold 검증
    for month in range(12):
        
        # 인덱스 설정
        test_indices = unique_monthly_indices[month]
        train_indices = []
        
        for month_idx in range(12):
            if month_idx != month:
                train_indices.extend(unique_monthly_indices[month_idx])

        # 이중 리스트를 하나의 리스트로 풀어내기
        train_flatten_indices = list(itertools.chain(*train_indices))
        test_flatten_indices = list(itertools.chain(*test_indices))
    
        print("test_indices{}".format(test_flatten_indices))
        
        X_test = cleaned_X[test_flatten_indices, :, :]
        y_test = cleaned_y[test_flatten_indices, :, :]
        X_train = cleaned_X[train_flatten_indices, :, :]
        y_train = cleaned_y[train_flatten_indices, :, :]

        # 예측 실행 전 데이터 존재 여부 확인 (월 전체가 결측치인 경우)
        if X_test.size == 0:
            print("No data available for predictions.")
            continue
            
        # u,v를 이어붙여서 모양 재생성
        #y_test_reshaped = y_test.reshape(y_test.shape[0], -1)
        #y_train_reshaped = y_train.reshape(y_train.shape[0], -1)

        # train, validation data split
        X_train_reshaped, X_val_reshaped, y_train_reshaped, y_val_reshaped = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42, shuffle=False)
        
        # 2차원 배열로 변환
        X_test_reshaped = X_test.reshape(X_test.shape[0] * X_test.shape[1], X_test.shape[2])
        X_val_reshaped = X_val_reshaped.reshape(X_val_reshaped.shape[0] * X_val_reshaped.shape[1], X_val_reshaped.shape[2])
        X_train_reshaped = X_train_reshaped.reshape(X_train_reshaped.shape[0] * X_train_reshaped.shape[1], X_train_reshaped.shape[2])
        
        # 스케일러 초기화 및 훈련 데이터에 대해 fit
        scaler = MaxAbsScaler()
        X_train_reshaped = scaler.fit_transform(X_train_reshaped)
        X_val_reshaped = scaler.transform(X_val_reshaped)
        X_test_reshaped = scaler.transform(X_test_reshaped)
        
        # 다시 3차원 배열로 변환
        X_train_reshaped = X_train_reshaped.reshape(-1, 24, 8)
        X_val_reshaped = X_val_reshaped.reshape(-1, 24, 8)
        X_test_reshaped = X_test_reshaped.reshape(-1, 24, 8)
        
        print("print test, train, validation shape")
        print(X_test_reshaped.shape)
        print(y_test.shape)
        print(X_train_reshaped.shape)
        print(y_train_reshaped.shape)
        print(X_val_reshaped.shape)
        print(y_val_reshaped.shape)

        # 입력 데이터의 형태
        input_steps = 24
        features = 8
        output_steps = 48  # 예측하고자 하는 출력의 time_steps
        output_features = 2  # 예측하고자 하는 출력의 feature 수
        CONV_WIDTH = 2
        STRIDE = 1
        model = build_regression_model(input_steps, features, output_steps, output_features, CONV_WIDTH, STRIDE)
        
        # 조기 종료 및 학습률 감소 콜백 설정
        early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
        
        # 모델 학습 및 손실 추적
        history = model.fit(X_train_reshaped, y_train_reshaped, epochs=400, batch_size=128,
                            validation_data=(X_val_reshaped, y_val_reshaped), 
                            callbacks=early_stopping)
        
        y_pred_reshaped = model.predict(X_test_reshaped)
        
        
        # 손실 추이 저장
        fold_train_loss = history.history['loss']
        fold_val_loss = history.history['val_loss']
        fold_test_loss, fold_test_mse = model.evaluate(X_test_reshaped, y_test, verbose=0)

        print(f'fold_test_mse: {fold_test_mse}')

        # RMSE 계산 및 저장
        fold_rmse = np.sqrt(fold_test_loss)
        fold_rmse_values.append(fold_rmse)

        print(f'Month {month + 1}: MSE = {fold_test_loss}, RMSE = {fold_rmse}')
        
        # 모든 폴드의 손실 추이 저장
        fold_train_losses.append(fold_train_loss)
        fold_val_losses.append(fold_val_loss)
        fold_test_losses.append(fold_test_loss)
            
        all_loss.extend(history.history['loss'])
        all_val_loss.extend(history.history['val_loss'])
    
        print('------------------------------------')
        print("y_pred, y_test shape")
        print(y_pred_reshaped.shape)
        print(y_test.shape)
        print('------------------------------------')

        # 각 폴드의 결과를 48개 그룹으로 나누어 저장
        for i in range(0,48):
            if i == 1:
                print(y_pred_reshaped)
                print(y_pred_reshaped[:, i, :])
                print(y_test[:, i, :])
            results_dict[i]['predictions'].extend(y_pred_reshaped[:, i, :])
            results_dict[i]['true_values'].extend(y_test[:, i, :])
            
            ''' if i ==0: # 0번째 행이 잘 추출되는지 확인 완료
                print(results_dict[i]['predictions'])
                print(results_dict[i]['true_values']) '''
        
        # 각 폴드에 대한 손실 추이 그래프 그리기
        plot_loss_and_rmse_graphs(history, fold_test_loss, month)
    
    fold_average = np.mean(fold_rmse_values)
    print(f'fold average: {fold_average}')
    print(len(results_dict[1]['predictions']))
    print(len(results_dict[1]['true_values']))
    
    model.summary()
    
    # 모델 구조를 시각적으로 저장
    plot_model(model, to_file='combined_model_structure.png', show_shapes=True, show_layer_names=True)

    # 모델 구조를 화면에 출력 (Jupyter Notebook에서)
    Image('combined_model_structure.png')

    return results_dict, fold_rmse_values

In [189]:
import matplotlib.font_manager as fm

# 글리프 8722를 포함하는 폰트를 설정
font_path = fm.findfont(fm.FontProperties(family='DejaVu Sans'))
plt.rcParams['font.sans-serif'] = [font_path]
plt.rcParams['axes.unicode_minus'] = False

def plot_groupwise_scatter(results_dict):
    # 전체 데이터 수집을 위한 리스트
    all_predictions = []
    all_true_values = []

    for group, data in results_dict.items():
        predictions = np.array(data['predictions']).flatten()
        true_values = np.array(data['true_values']).flatten()

        plt.figure(figsize=(4, 4))
        plt.scatter(true_values, predictions, alpha=0.5, label=f'Group {group}', c='blue')
        plt.scatter(true_values, true_values, alpha=0.5, label='True Values', c='red')
        plt.plot([true_values.min(), true_values.max()], [true_values.min(), true_values.max()], 'k--', lw=2)
        plt.xlabel('True Values')
        plt.ylabel('Predictions')
        plt.legend()
        plt.grid(True)
        plt.title(f'Predictions vs True Values (Group {group})')
        plt.tight_layout()
        plt.show()

        all_predictions.extend(predictions)
        all_true_values.extend(true_values)

    # 전체 데이터에 대한 산점도
    all_predictions = np.array(all_predictions).flatten()
    all_true_values = np.array(all_true_values).flatten()

    plt.figure(figsize=(8, 8))
    plt.scatter(all_true_values, all_predictions, alpha=0.5, label='Predictions', c='blue')
    plt.scatter(all_true_values, all_true_values, alpha=0.5, label='True Values', c='red')
    plt.plot([all_true_values.min(), all_true_values.max()], [all_true_values.min(), all_true_values.max()], 'k--', lw=2)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.legend()
    plt.grid(True)
    plt.title('Predictions vs True Values (All Groups)')
    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

# RMSE 계산 함수
def calculate_groupwise_rmse(results):
    groupwise_rmse = {}

    for group_key, values in results.items():
            
        predictions = np.vstack(values['predictions'])
        true_values = np.vstack(values['true_values'])
        if group_key == 0:
            print(predictions)
            print(true_values)
            print(len(predictions))
            print(len(true_values))
        print("pred_length: {}".format(len(predictions)))
        print("true_length: {}".format(len(true_values)))
        
        # MSE 및 RMSE 계산
        mse = mean_squared_error(true_values, predictions, multioutput='uniform_average')
        rmse = np.sqrt(mse)
        
        # 소수점 4자리까지 반올림
        groupwise_rmse[group_key] = round(rmse, 4)

    return groupwise_rmse

# 결과 딕셔너리에서 모든 그룹의 RMSE 계산
results, rmse_values = calculate_and_store_results(X, y_processed, monthly_indices)
groupwise_rmse = calculate_groupwise_rmse(results)

In [ ]:
# 결과 출력
for group_key, rmse in groupwise_rmse.items():
    print(f"RMSE for {group_key}: {rmse:.4f}")

In [192]:
df = pd.DataFrame(list(groupwise_rmse.values()), columns=['RMSE'])

In [193]:
df = df.transpose()

In [ ]:
df

In [ ]:
df.iloc[0].mean()

In [ ]:
df.iloc[0, :24].mean()

In [176]:
#df.to_excel('./rmse_s3_robust.xlsx')